# Welch Algorithm

A small factory consists of a machining center and inspection station
in series. Unfinished parts arrive to the factory with exponential
interarrival times having a mean of 1 minute. Processing times at the machine are uni-
form on the interval [0.65, 0.70] minute, and subsequent inspection times at the inspection
station are uniform on the interval [0.75, 0.80] minute. Ninety percent of inspected parts
are “good” and are sent to shipping; 10 percent of the parts are “bad” and are sent back
to the machine for rework. (Both queues are assumed to have infinite capacity.) The
machining center is subject to randomly occurring breakdowns. In particular, a new (or
freshly repaired) machine will break down after an exponential amount of calendar
time with a mean of 6 hours. Repair times are uniform on the interval
[8, 12] minutes. If a part is being processed when the machine breaks down, then the
machine continues where it left off upon the completion of repair. Assume that the factory
is initially empty and idle, and is open 8 hours per day.
Consider the stochastic process N1, N2, . . . , where Ni is the number of parts pro-
duced in the ith hour.


In [1]:
import simpy
import random

class Factory:
    def __init__(self, env):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)  # Machining center
        self.inspection_station = simpy.Resource(env, capacity=1)  # Inspection station
        self.parts_produced = 0
        self.breakdown_event = env.event()

    def machining(self, part_id):
        with self.machine.request() as req:
            yield req
            machining_time = random.uniform(MACHINE_TIME_LOW, MACHINE_TIME_HIGH)
            try:
                print(f"Part {part_id}: starts machining at {env.now:.2f}")
                yield env.timeout(machining_time)
                print(f"Part {part_id}: finishes machining at {env.now:.2f}")
            except simpy.Interrupt:
                print(f"Part {part_id}: machine breakdown during machining at {env.now:.2f}")
                repair_time = random.uniform(REPAIR_TIME_LOW, REPAIR_TIME_HIGH)
                yield env.timeout(repair_time)
                print(f"Part {part_id}: machine repaired, resuming machining at {env.now:.2f}")
                yield env.timeout(machining_time - (env.now - self.env.now))

    def inspect(self, part_id):
        with self.inspection_station.request() as req:
            yield req
            inspection_time = random.uniform(INSPECTION_TIME_LOW, INSPECTION_TIME_HIGH)
            print(f"Part {part_id}: starts inspection at {env.now:.2f}")
            yield env.timeout(inspection_time)
            print(f"Part {part_id}: finishes inspection at {env.now:.2f}")

    def process_part(self, part_id):
        # Machining step
        yield env.process(self.machining(part_id))
        
        # Inspection step
        yield env.process(self.inspect(part_id))
        
        # Determine if the part needs rework
        if random.random() < REWORK_PROBABILITY:
            print(f"Part {part_id}: sent back for rework at {env.now:.2f}")
            yield env.process(self.machining(part_id))
            yield env.process(self.inspect(part_id))
        
        # Part is finally good
        self.parts_produced += 1
        print(f"Part {part_id}: good part produced at {env.now:.2f}")

def arrival_process(env, factory):
    part_id = 0
    while True:
        interarrival_time = random.expovariate(1.0 / MEAN_INTERARRIVAL)
        yield env.timeout(interarrival_time)
        part_id += 1
        env.process(factory.process_part(part_id))

def machine_breakdown(env, factory):
    while True:
        breakdown_time = random.expovariate(1.0 / BREAKDOWN_MEAN)
        yield env.timeout(breakdown_time)
        print(f"Machine breaks down at {env.now:.2f}")
        factory.breakdown_event.succeed()
        factory.breakdown_event = env.event()

def track_hourly_production(env, factory):
    while True:
        yield env.timeout(60)  # 1 hour
        hourly_production.append(factory.parts_produced)
        print(f"Hour {len(hourly_production)}: {factory.parts_produced} parts produced.")
        factory.parts_produced = 0  # Reset for the next hour


In [2]:
# Define constants
MEAN_INTERARRIVAL = 1.0  # Mean interarrival time in minutes
MACHINE_TIME_LOW = 0.65  # Min machine processing time
MACHINE_TIME_HIGH = 0.70  # Max machine processing time
INSPECTION_TIME_LOW = 0.75  # Min inspection time
INSPECTION_TIME_HIGH = 0.80  # Max inspection time
REWORK_PROBABILITY = 0.10  # Probability that a part is bad and requires rework
BREAKDOWN_MEAN = 360  # Mean time for machine breakdown in minutes
REPAIR_TIME_LOW = 8  # Min repair time in minutes
REPAIR_TIME_HIGH = 12  # Max repair time in minutes
WORK_DAY = 8 * 60  # Factory workday length in minutes (8 hours)

# Global variables to track the number of parts produced each hour
hourly_production = []

# Set up the simulation
random.seed(42)  # For reproducibility
env = simpy.Environment()

# Initialize the factory
factory = Factory(env)

# Start processes
env.process(arrival_process(env, factory))
env.process(machine_breakdown(env, factory))
env.process(track_hourly_production(env, factory))

# Run the simulation for 8 hours (480 minutes)
env.run(until=WORK_DAY)

# Print final results
print("Hourly production counts:", hourly_production)

Part 1: starts machining at 1.02
Part 1: finishes machining at 1.68
Part 2: starts machining at 1.68
Part 1: starts inspection at 1.68
Part 2: finishes machining at 2.37
Part 1: finishes inspection at 2.48
Part 1: sent back for rework at 2.48
Part 2: starts inspection at 2.48
Part 1: starts machining at 2.48
Part 1: finishes machining at 3.13
Part 3: starts machining at 3.13
Part 2: finishes inspection at 3.25
Part 2: good part produced at 3.25
Part 1: starts inspection at 3.25
Part 3: finishes machining at 3.78
Part 4: starts machining at 3.78
Part 1: finishes inspection at 4.03
Part 1: good part produced at 4.03
Part 3: starts inspection at 4.03
Part 4: finishes machining at 4.44
Part 5: starts machining at 4.44
Part 3: finishes inspection at 4.81
Part 3: good part produced at 4.81
Part 4: starts inspection at 4.81
Part 5: finishes machining at 5.09
Part 6: starts machining at 5.09
Part 4: finishes inspection at 5.59
Part 4: good part produced at 5.59
Part 5: starts inspection at 5.5